In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../src/')
import dataset_prep

Import de la base de données

In [2]:
selected_columns = [
"nb_packets_sent",
"distance",
"distRealSR1",
"pos_y_rec_f",
"pos_y_rec",
"pos_x_rec_f",
"pos_x_rec",
"pos_x_send",
"pos_y_send",
"spd_x_send",
"spd_y_send",
"sendTime",
"receiver",
"sender",
"label"
]

In [3]:
df = dataset_prep.import_dataset('../data/DoS_0709_new_columns.csv', selected_columns)

In [4]:
dataset_prep.clean_dataset(df)

Nombre de lignes avant nettoyage :  4753433
Nombre de lignes après nettoyage :  4709293


Analyse et vérification des données

In [5]:
senders = np.unique(df["sender"].values)
senders

array([    9,    15,    21, ..., 24399, 24405, 24411], dtype=int64)

In [6]:
print(len(senders))

4067


In [7]:
df.loc[df['sender'] == senders[0]].sort_values("sendTime")

,sendTime,sender,receiver,pos_x_send,pos_y_send,spd_x_send,spd_y_send,label,pos_x_rec,pos_y_rec,pos_x_rec_f,pos_y_rec_f,nb_packets_sent,distRealSR1,distance
4,25212.602877,9,45,257.604242,141.917136,-2.409908,14.316189,0,265.540271,46.207434,268.539855,49.801087,96,7.936029,96.038156
5,25213.602877,9,45,255.242943,156.206787,-2.566640,14.210027,0,268.539855,49.801087,268.050977,55.331302,96,13.296912,107.233302
6,25214.602878,9,45,252.857751,170.495740,-2.342829,14.254332,0,268.050977,55.331302,270.113953,63.318293,96,15.193227,116.162309
7,25215.602879,9,45,250.525544,184.832952,-2.352874,14.315002,0,270.113953,63.318293,268.836896,72.915064,96,19.588409,123.083379
8,25216.602880,9,45,248.511212,199.166902,-2.079440,14.360857,0,268.836896,72.915064,267.267270,84.617270,96,20.325683,127.877519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,25291.602880,9,225,134.821280,1024.506204,-0.602227,8.137469,0,138.085770,1102.386398,138.801402,1096.647173,96,3.264490,77.948582
65,25291.602880,9,147,134.821280,1024.506204,-0.602227,8.137469,0,147.593919,898.797866,147.021568,894.975244,96,12.772639,126.355556
91,25292.602869,9,219,134.539820,1030.197090,-0.270067,3.653434,0,142.457918,1043.790636,143.221108,1033.675922,96,7.918098,15.731521
95,25292.602869,9,225,134.539820,1030.197090,-0.270067,3.653434,0,138.801402,1096.647173,139.551488,1089.075886,96,4.261582,66.586594


In [8]:
# Copie de la bdd pour récuperer les colonnes sans impacter la vraie base de données

data_copy = df.copy()
data_copy = data_copy.drop(["label"], axis=1)

columns = data_copy.columns

Amélioration à faire :

Séléctionner seulement les sender avec value_counts > 20

Séparation de la base de données : diviser pour régner

In [9]:
# /!\ attention à bien utiliser la base de données rangée par sender pour limiter la perte de données
# Indice égal à 0, 1 ou 2
# Célia = 0
# Chléo = 1
# Thomas = 2

def split_data_in_3(df, indice):
    # Division de la bdd en 3
    split_data = np.array_split(df,3)

    # Division en plusieurs sous-bdd pour diminuer le temps de calcul par exécution
    #divided_data = np.array_split(split_data[indice],50)

    return split_data

/!\ à bien changer l'indice !!

In [10]:
divided_dataframes = split_data_in_3(df.sort_values("sender"),0)

In [11]:
len(divided_dataframes)

3

In [ ]:
sequence = (df.loc[df['sender'] == senders[0]].sort_values("sendTime"))[0:20]
sequence.values.tolist()

In [13]:
df.loc[df['receiver'] == senders[0]]['label'].tolist()[0]

0

Création des séquences

In [14]:
def sequence_creation(df):
    senders_sequences = []
    senders_label = []
    for sender in senders:
        sender_data_sorted = df.loc[df['sender'] == sender].sort_values("sendTime")
        sender_data_sorted = sender_data_sorted.drop(["label","sender"], axis=1)
        sequence_array = []

        length = df.loc[df['receiver'] == sender].shape[0]
        slide = 10
        start = 0
        end = 20

        # On récupère la valeur du label pour ce sender
        label = df.loc[df['receiver'] == senders[0]]['label'].tolist()[0]
        if (length > 20):
            senders_label.append(label)

        # On vérifie qu'il est possible de faire une séquence de taille 20
        while length > 20:
            # Extraction par tranche de 20 avec une inter de 10
            sequence = sender_data_sorted[start:end]

            # Labels correspondant
            #labels =  pd.Series.tolist(sequence["label"])

            # On transforme les 13 en 1, cette formule marche toujours si on met d'autres types d'attaques
            #labels[:] = [x if x == 0 else 1 for x in labels]

            # Attribution des tableaux numpy
            sequence_array.append(sequence.values.tolist())
            #label_array.append(labels)

            # Mise à jour des variables
            start += slide
            end += slide
            length -= 10
        
        if len(sequence_array)>0:
            senders_sequences.append(sequence_array)

    return senders_sequences, senders_label

/!\ à bien changer l'indice !!

In [15]:
sequence_test, label_test = sequence_creation(divided_dataframes[0])

Vérification des séquences obtenues

In [156]:
len(sequence_test)

1699

Test RNN

In [17]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import layers, applications, utils, models, optimizers, Input, callbacks
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

In [157]:
# Transformation en array numpy
X = sequence_test
y = np.array(label_test)

In [158]:
# Séparation en données d'entrainement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [161]:
# print("X_train : ", X_train.shape)
print("y_train : ", y_train.shape)
# print("X_test : ", X_test.shape)
print("y_test : ", y_test.shape)

y_train :  (1529,)
y_test :  (170,)


In [117]:
X_flat = [item for sitem in X_train[:1][0] for item in sitem]
# print(X_flat[:20])


In [167]:
# X_train = np.asarray(X_train[:20])
X_train = np.asarray(X_flat[:1000])
y_train = np.asarray(y_train[:1000])

In [ ]:
print(X_train)

In [168]:
len(X_train)

1000

In [169]:
simple_model = keras.Sequential([
    layers.Dense(32,input_shape=(len(X_train[0]),), activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [170]:
callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

In [171]:
opt = keras.optimizers.SGD(learning_rate=0.1)
simple_model.compile(loss='mean_squared_error', optimizer='adam', metrics='accuracy')

In [172]:
history = simple_model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=2)

Epoch 1/10
8/8 - 2s - loss: 0.0949 - accuracy: 0.8590 - 2s/epoch - 238ms/step
Epoch 2/10
8/8 - 0s - loss: 9.3010e-05 - accuracy: 1.0000 - 29ms/epoch - 4ms/step
Epoch 3/10
8/8 - 0s - loss: 1.4270e-05 - accuracy: 1.0000 - 28ms/epoch - 3ms/step
Epoch 4/10
8/8 - 0s - loss: 6.8008e-06 - accuracy: 1.0000 - 31ms/epoch - 4ms/step
Epoch 5/10
8/8 - 0s - loss: 5.6230e-06 - accuracy: 1.0000 - 27ms/epoch - 3ms/step
Epoch 6/10
8/8 - 0s - loss: 3.7496e-06 - accuracy: 1.0000 - 25ms/epoch - 3ms/step
Epoch 7/10
8/8 - 0s - loss: 4.4411e-06 - accuracy: 1.0000 - 26ms/epoch - 3ms/step
Epoch 8/10
8/8 - 0s - loss: 3.7759e-06 - accuracy: 1.0000 - 33ms/epoch - 4ms/step
Epoch 9/10
8/8 - 0s - loss: 4.1812e-06 - accuracy: 1.0000 - 31ms/epoch - 4ms/step
Epoch 10/10
8/8 - 0s - loss: 3.2074e-06 - accuracy: 1.0000 - 31ms/epoch - 4ms/step
